#Install Blender and some required libraries (README)
Before that you need to mount your Google Drive.
 
##How to mount Google Drive
 
###On desktop:
Click the folder icon on left side vertical bar, then click on the folder with symbol of Google Drive
 
###On mobile:
Open side navigation and click 'Show file browser', then click on the folder with the symbol of Google Drive
 
Then goto Edit, Notebook Settings and select Hardware Accelerator as GPU and save it, else it will not work.
After that run the first cell to install Blender.
 
##NOTE: Notebooks run by connecting to virtual machines that have maximum lifetimes that can be as much as 12 hours. Notebooks will also disconnect from VMs when left idle for too long and you won't be able to use GPU for a while. After rendering is completed, go to 'Runtime', then 'Manage Session' and terminate the session by clicking on the dustbin button along with current running session.

In [ ]:
#@title Alternatively, you can mount Google Drive using this cell (optional if you already did via above method) { display-mode: "form" }
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Install Blender and required libraries { display-mode: "form" }
 
#Downloads Blender
!wget https://download.blender.org/release/Blender2.91/blender-2.91.0-linux64.tar.xz
 
#Installs Blender
!tar xf blender-2.91.0-linux64.tar.xz
!rm blender-2.91.0-linux64.tar.xz
 
#Download some required libraries
!apt install libboost-all-dev
!apt install libgl1-mesa-dev
 
f=open("set_gpu.py", "w+")
f.write("""
import bpy
def enable_gpus(device_type, use_cpus=False):
    preferences = bpy.context.preferences
    cycles_preferences = preferences.addons["cycles"].preferences
    cuda_devices, opencl_devices = cycles_preferences.get_devices()
 
    if device_type == "CUDA":
        devices = cuda_devices
    elif device_type == "OPENCL":
        devices = opencl_devices
    else:
        raise RuntimeError("Unsupported device type")
 
    activated_gpus = []
 
    for device in devices:
        if device.type == "CPU":
            device.use = use_cpus
        else:
            device.use = True
            activated_gpus.append(device.name)
 
    cycles_preferences.compute_device_type = device_type
    bpy.context.scene.cycles.device = "GPU"
 
    return activated_gpus
enable_gpus("CUDA")
""")
f.close()

#Filepath and Output
##Enter the path for input and output
 
For file_path: mention the folder where the .blend file is stored. 
 
Example: Blend/file.blend
 
 
_(You don't need to write drive/MyDrive/Blend/file since it is written in the code)_
 
 
For out_path: mention the folder (with filename [optional])
 
Example: Blend/output_foldername/filename (no extension like .jpg or .png)

In [ ]:
#@title Blender Cycles Render { display-mode: "form" }
 
#@markdown ### Enter the file path:
file_path = "Blend/someroom.blend" #@param {type:"string"}
filename = "drive/MyDrive/"+file_path
print('Input directory: ',filename)
#@markdown ### Enter the file path:
out_path = "Blend/output/" #@param {type:"string"}
output = "drive/MyDrive/"+out_path
print('Output directory: ',output)

#Select file type for single frame render
First select file output type from the dropdown and run 'Output Type' cell. After that run 'Start Rendering' cell.

In [ ]:
#@title Output Type { display-mode: "form" }
 
otype = "PNG" #@param ["PNG","JPEG", "BMP", "IRIS", "IRIZ", "AVIRAW", "TGA", "RAWTGA", "AVIJPEG"]
print("Selected: ",otype)
 
frame_no = 1 #@param {type:"number"}

In [ ]:
#@title Start Rendering { display-mode: "form" }
#@markdown ###Note: After the render is finished, the output will be saved in the output directory mentioned above. After that Blender will be uninstalled for saving space
 
!sudo ./blender-2.91.0-linux64/blender -b $filename -noaudio -P ./set_gpu.py -o $output -E 'CYCLES' -f $frame_no -F $otype
#This will uninstall Blender after the render
!rm -rf ./blender-2.91.0-linux64
!rm set_gpu.py